# Windmill Problem Visualization 

In [58]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [59]:
%matplotlib tk

### Config

In [60]:
N = 3  # number of points
K = 1000 # number of steps in one windmill rotation
T = 10   # seconds per windmill rotation

### Pick Random Points

In [61]:
ptx = np.random.rand(N)
pty = np.random.rand(N)

In [62]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
line, = ax.plot([], [], 'r-')
ax.plot(ptx,pty,'ko')
ax.set_xlim([0, 1]); ax.set_ylim([0, 1])
ax.set_xticks([]); ax.set_yticks([])

[]

### Propagate Pivot Point and Angle
- `p` stores index of active pivot
- `t` is angle in radians

In [63]:
p = np.zeros(shape=(K), dtype=int)
t = np.linspace(-np.pi, np.pi, K, endpoint=False)
dt = 2 * np.pi / K

# get headings between each point [-pi,pi]
dx = ptx.reshape(1, N) - ptx.reshape(N, 1)
dy = pty.reshape(1, N) - pty.reshape(N, 1)
headings = np.arctan2(dy, dx)

# ignore diagonal
for i in range(N):
    headings[i, i] = np.inf
    
# map angles to [-pi, pi)
def ang(a):
    return (a + np.pi) % (2 * np.pi) - np.pi
    
i = 0
while i < K:
    # get the current angle of front side of line, and the headings to each point
    tc = t[i]
    hc = headings[p[i], :]
    
    # get the angle of rotation to reach each point for the front and back side of line
    front_diff = ang(hc - tc)
    back_diff = ang(hc - tc - np.pi)
    
    # get the next point that the front side and back side of line will hit
    next_front = np.where(front_diff > 0, front_diff, np.inf).argmin()
    next_back = np.where(back_diff > 0, back_diff, np.inf).argmin()
    
    # pick whichever point we'll hit first,and how long we travel to get there
    if front_diff[next_front] < back_diff[next_back]:
        nextp = next_front
        delta = front_diff[nextp]
    else:
        nextp = next_back
        delta = back_diff[nextp]
        
    # how many steps to get there
    num = int(np.ceil(delta / dt))
    
    # don't go past end
    if i + num >= K:
        p[i:] = p[i]
        i = K
    else:
        p[i:i+num] = p[i]
        p[i+num] = nextp
        i += num

/home/peter/Projects/Windmill/venv/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in remainder
  app.launch_new_instance()


### Convert Point and Angle to Plotting Points 

In [64]:
x = np.zeros(shape=(K, 2))
y = np.zeros(shape=(K, 2))

for i in range(K):
    m = np.tan(t[i])
    px0 = ptx[p[i]]
    py0 = pty[p[i]]
    
    if abs(m) > 1:
        y[i, :] = [-1, 1]
        x[i, 0] = (-1 - py0) / m + px0
        x[i, 1] = (1 - py0) / m + px0
    else:
        x[i, :] = [-1, 1]
        y[i, 0] = m * (-1 - px0) + py0
        y[i, 1] = m * (1 - px0) + py0

### Animate 

In [65]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
line, = ax.plot([], [], 'r-')
ax.plot(ptx,pty,'ko')
ax.set_xlim([0, 1]); ax.set_ylim([0, 1])
ax.set_xticks([]); ax.set_yticks([])

def update(i):
    line.set_data(x[i % K, :], y[i % K, :])
    return line

ani = FuncAnimation(fig, update, interval=1000 * T / K)

plt.show()